#Titanic: Machine Learning from Disaster
Olivier RISSER-MAROIX (VieVie31)

In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
data_train = gl.load_sframe("train.csv")
data_test = gl.load_sframe("test.csv")

2016-03-18 16:31:48,465 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458315104.log


Finished parsing file /Users/mac/Desktop/titanic/train.csv

Parsing completed. Parsed 100 lines in 0.199079 secs.

This non-commercial license of GraphLab Create is assigned to orissermaroix@gmail.com and will expire on March 12, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /Users/mac/Desktop/titanic/train.csv

...

Parsing completed. Parsed 100 lines in 0.007563 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /Users/mac/Desktop/titanic/test.csv

Parsing completed. Parsed 418 lines in 0.008804 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Pclass	int
	Name	str
	Sex	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	Embarked	str

Rows: 3

Data:
+-------------+----------+--------+-------------------------------+--------+
| PassengerId | Survived | Pclass |              Name             |  Sex   |
+-------------+----------+--------+-------------------------------+--------+
|      1      |    0     |   3    |    Braund, Mr. Owen Harris    |  male  |
|      2      |    1     |   1    | Cumings, Mrs. John Bradley... | female |
|      3      |    1     |   3    |     Heikkinen, Miss. Laina    | female |
+-------------+----------+--------+-------------------------------+--------+
+------+-------+-------+------------------+---------+-------+----------+
| Age  | SibSp | Parch |      Ticket      |   Fare  | Cabin | Embarked |
+------+-------+-------+------------------+---------+-------+----------+
| 22.0 |   1   |   0   |    A/5 21171     |   7.25  |       |    S     |
| 38.0 |   1   |   0   |     PC 17599     | 71.2833 |  C85  |    C     |
| 26.0 |   0   |   0   | STON/O2. 3101282 |  7.925  |       |    S     |
+------+-------+-------+------------------+---------+-------+----------+
[3 rows x 12 columns]

##Cleanning trainning data

In [4]:
data_train["male"] = data_train["Sex"] == "male"
data_train["female"] = data_train["Sex"] == "female"
data_train = data_train.remove_column("Sex")

In [5]:
data_train["no_age"] = data_train["Age"] == None
data_train["Age"] = gl.SArray([0 if v == None else v for v in data_train["Age"]])

In [6]:
data_train["embarked_s"] = data_train["Embarked"] == "S"
data_train["embarked_c"] = data_train["Embarked"] == "C"
data_train["embarked_q"] = data_train["Embarked"] == "Q"
data_train["embarked_none"] = data_train["Embarked"] == None
data_train = data_train.remove_column("Embarked")

In [7]:
data_train["1_class"] = data_train["Pclass"] == 1
data_train["2_class"] = data_train["Pclass"] == 2
data_train["3_class"] = data_train["Pclass"] == 3
data_train = data_train.remove_column("Pclass")

##Create logistic model

In [8]:
#help(gl.classifier.logistic_classifier.create)

In [9]:
train_set_1, train_set_2 = data_train.random_split(.8)

In [10]:
print train_set_1.head(1)
features = ["Age", "SibSp", "Parch", "Fare", "male", "female", "no_age", 
            "embarked_s", "embarked_c", "embarked_q", "embarked_none",
            "1_class", "2_class", "3_class"]

+-------------+----------+-------------------------+------+-------+-------+
| PassengerId | Survived |           Name          | Age  | SibSp | Parch |
+-------------+----------+-------------------------+------+-------+-------+
|      1      |    0     | Braund, Mr. Owen Harris | 22.0 |   1   |   0   |
+-------------+----------+-------------------------+------+-------+-------+
+-----------+------+-------+------+--------+--------+------------+------------+
|   Ticket  | Fare | Cabin | male | female | no_age | embarked_s | embarked_c |
+-----------+------+-------+------+--------+--------+------------+------------+
| A/5 21171 | 7.25 |       |  1   |   0    |   0    |     1      |     0      |
+-----------+------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+
| embarked_q | embarked_none | 1_class | 2_class | 3_class |
+------------+---------------+---------+---------+---------+
|     0      |       0       |    0  

In [11]:
simple_logistic_classifier = gl.classifier.logistic_classifier.create(train_set_1, target="Survived", 
                                                                      features=features, validation_set=train_set_2)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 708

Number of classes           : 2

...

| 2         | 3        | 1.218790     | 0.809322          | 0.781421            |

| 3         | 4        | 1.222348     | 0.810734          | 0.781421            |

| 4         | 5        | 1.225361     | 0.812147          | 0.781421            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create SVM model

In [14]:
simple_svm_classifier = gl.classifier.svm_classifier.create(train_set_1, target="Survived", 
                                                            features=features, validation_set=train_set_2, 
                                                            max_iterations=100)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

SVM:

--------------------------------------------------------

Number of examples          : 708

Number of classes           : 2

...

| 5         | 8        | 1.000000  | 0.016146     | 0.789548          | 0.786885            |

| 6         | 9        | 1.000000  | 0.018314     | 0.810734          | 0.797814            |

| 11        | 14       | 1.000000  | 0.028337     | 0.800847          | 0.792350            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create a decision tree model

In [25]:
decision_tree_model = gl.decision_tree_classifier.create(train_set_1, validation_set=train_set_2,
                                                               target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 708

Number of classes           : 2

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.119060     | 0.879943          | 0.546775          | 0.792350            | 0.580433            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Boosted Tree model

In [31]:
boosted_tree_model = gl.classifier.boosted_trees_classifier.create(train_set_1, validation_set=train_set_2,
                                                                   target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 708

Number of classes           : 2

...

| 3         | 0.018047     | 0.885593          | 0.408435          | 0.786885            | 0.493694            |

| 4         | 0.021902     | 0.891243          | 0.371841          | 0.797814            | 0.478705            |

| 5         | 0.026236     | 0.896893          | 0.341849          | 0.803279            | 0.466879            |

| 6         | 0.031464     | 0.899718          | 0.320911          | 0.792350            | 0.468996            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Cleanning testing data

In [15]:
data_test["male"] = data_test["Sex"] == "male"
data_test["female"] = data_test["Sex"] == "female"
data_test = data_test.remove_column("Sex")

In [16]:
data_test["no_age"] = data_test["Age"] == None
data_test["Age"] = gl.SArray([0 if v == None else v for v in data_test["Age"]])

In [17]:
data_test["embarked_s"] = data_test["Embarked"] == "S"
data_test["embarked_c"] = data_test["Embarked"] == "C"
data_test["embarked_q"] = data_test["Embarked"] == "Q"
data_test["embarked_none"] = data_test["Embarked"] == None
data_test = data_test.remove_column("Embarked")

In [18]:
data_test["1_class"] = data_test["Pclass"] == 1
data_test["2_class"] = data_test["Pclass"] == 2
data_test["3_class"] = data_test["Pclass"] == 3
data_test = data_test.remove_column("Pclass")

##Making Predictions

In [32]:
data_test["Survived"] = boosted_tree_model.predict(data_test)
#decision_tree_model.predict(data_test)
#simple_svm_classifier.predict(data_test) 
#simple_logistic_classifier.predict(data_test)

In [33]:
submission = gl.SFrame()

In [34]:
submission["PassengerId"] = data_test["PassengerId"]
submission["Survived"] = data_test["Survived"]

In [35]:
submission.save("kaggle.csv", format="csv")